In [1]:
import pandas as pd
import os

First step is to create the connection to Databricks

In [2]:
# Connect to Databricks
from databricks import sql
import os
from dotenv import load_dotenv
from pyspark.sql import SparkSession
import pandas as pd

# Load the environment variables
load_dotenv()

# Set the connection
connection =  sql.connect(server_hostname = os.getenv("DATABRICKS_SERVER_HOSTNAME"),
                 http_path       = os.getenv("DATABRICKS_HTTP_PATH"),
                 access_token    = os.getenv("PYTHON_ACCESS_TOKEN"))

cursor = connection.cursor()


# Start spark session
spark = SparkSession.builder \
    .appName("mkt_analysis_2024") \
    .getOrCreate()

def get_df_from_databricks(query):
    # Execute the query
    cursor.execute(query)
    # Get the result
    result = cursor.fetchall()
    # Get the column names
    columns = [desc[0] for desc in cursor.description]
    # Create a pandas DataFrame
    df = pd.DataFrame(result, columns=columns)
    return df


24/05/18 16:21:14 WARN Utils: Your hostname, Bernardos-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.68.117 instead (on interface en0)
24/05/18 16:21:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/18 16:21:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


A bit about how the data is structured at Metaloop. We essentially have these sources of trasactional material data:

 - Cart items
    - These are the materials that compose a transaction (stored in the orders object). Each order is associated with two accounts, a seller and a buyer. Therefore, the lowest granularity data here will be cart item level data with data from the orders joined
 - Opportunity materials
    - These paly a similar role as cart items, but on the opportunity level (prior to an order being put forward)
 - Conditions (Contract materials)

 

In [34]:
# Read the query from the query file    
queries_path = os.path.join(os.getcwd(), "queries/")

# Read the cart items query
query = open(queries_path + "cart_items.sql").read()

# Get cart_items
cart_items = get_df_from_databricks(query)

In [35]:
# Print the dataset
cart_items.head()

,pk_cart_item,fk_order,ds_material_alias,ds_material,fk_material_alias,fk_material,ds_purchase_price_formula,ds_sale_price_formula,vl_quantity_purchase,vl_quantity_sell,vl_unit_price_purchase,vl_unit_price_sell,ds_status_label
0,a0V09000000p8gCEAQ,a0b09000000gKg6AAE,Cu mit Fe (87%),"Kupfer, gemischt, Raff, alt",a0g09000001F4tuAAC,a0Z09000005ERBFEA4,None,None,374.00,374.00,4.380000,4.380000,Webshop
1,a0V09000000p8vyEAA,a0b09000000gKgBAAU,"Kupfer, blank I, Millberry","Kupfer, blank I, Millberry",a0g09000001F4tqAAC,a0Z09000000oVReEAM,None,None,220.00,220.00,5.150000,5.150000,Webshop
2,a0V09000000p8vzEAA,a0b09000000gKgBAAU,"Kupfer schwer, Berry min. 98% Cu","Kupfer schwer, Berry min. 98% Cu",a0g09000001F4tmAAC,a0Z09000005ERBKEA4,None,None,25.00,25.00,4.430000,4.430000,Webshop
3,a0V09000000p8w0EAA,a0b09000000gKgBAAU,Kupferkabel mit Stecker,Kupferkabel mit Stecker,a0g09000001F4uMAAS,a0Z09000005ESrsEAG,None,None,5.00,5.00,0.670000,0.670000,Webshop
4,a0V09000000p8w1EAA,a0b09000000gKgBAAU,"Kupferkabel, min. 38% Cu","Kupferkabel, min. 38% Cu",a0g09000001F4uOAAS,a0Z09000000oVSIEA2,None,None,50.00,50.00,1.410000,1.410000,Webshop


## Simple bag of words model

With a simple bag of words model, we try to extract features from the alias, material and formula fields. Our target variable is the variable product_id, which is a class corresponding to the underlying product related to the material. This is encoded by the class in fk_material column.
We will also only use the aliases at this point

In [36]:
# Revoving the unncessary columns
df1 = cart_items.drop(['pk_cart_item', 'fk_order', 'fk_material_alias', 'ds_status_label', 'ds_material'], 
                      axis=1)

# Change ds_material_alias to string
df1['ds_material_alias'] = df1['ds_material_alias'].astype(str)

Here we can actually "duplicate" the 

In [38]:
# Define function to count words
def count_words(string):
    return len(string.split())

# Define function to get the average word length
def avg_word_length(string):
    words = string.split()
    return sum(len(word) for word in words) / len(words)

# Check if the description has the percentage symbol
df1['flg_has_perc'] = df1['ds_material_alias'].apply(lambda x: len([c for c in x if c == '%'])>0)

# Create number of words for the two columns available
df1['n_words_alias'] = df1['ds_material_alias'].apply(count_words)

df1.head()


,ds_material_alias,fk_material,ds_purchase_price_formula,ds_sale_price_formula,vl_quantity_purchase,vl_quantity_sell,vl_unit_price_purchase,vl_unit_price_sell,n_words_alias,flg_has_perc
0,Cu mit Fe (87%),a0Z09000005ERBFEA4,None,None,374.00,374.00,4.380000,4.380000,4,True
1,"Kupfer, blank I, Millberry",a0Z09000000oVReEAM,None,None,220.00,220.00,5.150000,5.150000,4,False
2,"Kupfer schwer, Berry min. 98% Cu",a0Z09000005ERBKEA4,None,None,25.00,25.00,4.430000,4.430000,6,True
3,Kupferkabel mit Stecker,a0Z09000005ESrsEAG,None,None,5.00,5.00,0.670000,0.670000,3,False
4,"Kupferkabel, min. 38% Cu",a0Z09000000oVSIEA2,None,None,50.00,50.00,1.410000,1.410000,4,True


24/05/19 02:28:39 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 338553 ms exceeds timeout 120000 ms
24/05/19 02:28:39 WARN SparkContext: Killing executors is not supported by current scheduler.
24/05/19 02:28:46 WARN Executor: Issue communicating with driver in heartbeater
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:101)
	at org.apache.spark.rpc.RpcEndpointRef.askSync(RpcEndpointRef.scala:85)
	at org.apache.spark.storage.BlockManagerMaster.registerBlockManager(BlockManagerMaster.scala:80)
	at org.apache.spark.storage.BlockManager.reregister(BlockManager.scala:642)
	at org.apache.spark.executor.Executor.reportHeartBeat(Executor.scala:1223)
	at o

## Tokenization

1. Converting words into lowercase
2. Removing leading and trailing whitespaces
3. Removing punctuation
4. Removing stopwords
5. Expanding contractions
6. Removing special characters

Can we perform lemmatazation, given that a lot of our text is in German? Yes we can, check out the models at the [spaCy documentation](https://spacy.io/usage/models)

We will need an algorithm to determine in what language the text is (or is likely to be) to perform the best possible tokenization

Note, after loading the spacy package you might need to reboot your computer (the first time). If you are getting this error:
> ValueError: [E002] Can't find factory for 'transformer' for language English (en). This usually happens when spaCy calls `nlp.create_pipe` with a custom component name that's not registered on the current language class. If you're using a Transformer, make sure to install 'spacy-transformers'. If you're using a custom component, make sure you've added the decorator `@Language.component` (for function components) or `@Language.factory` (for class components).


In [1]:
# Tokenazation example with 'Kupfer, blank I, Millberry'
import spacy
import spacy_transformers
import de_dep_news_trf

# Load the en_core_web_sm model
de_nlp = spacy.load('de_dep_news_trf')

# Create a Doc object
doc = de_nlp('Kupfer, blank I, Millberry')

# Generate the tokens
tokens = [token.text for token in doc]
print(tokens)

/Users/bernardocarvalho/Desktop/imperial_repos/work_based_project/venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/bernardocarvalho/Desktop/imperial_repos/work_based_project/venv/lib/python3.11/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/Users/bernardocarvalho/Desktop/imperial_repos/work_based_project/venv/lib/python3.11/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


['Kupfer', ',', 'blank', 'I', ',', 'Millberry']


In [2]:
lemmas = [token.lemma_ for token in doc]

print(lemmas)


['Kupfer', '--', 'blank', 'I', '--', 'Millberry']


As part of the baseline exercise, we will use standard stopwords. However, as a potential improvement to build custom stopword lists tailored to this use-case


In [4]:
# Get german default stopwords
de_stopwords  = spacy.lang.de.stop_words.STOP_WORDS

# Get english default stopwords
en_stopwords = spacy.lang.en.stop_words.STOP_WORDS

In [ ]:
def german_text_tokenization(text):
    # Create a Doc object
    doc = de_nlp(text)
    # Generate the tokens
    tokens = [token.text for token in doc]
    return tokenized_text

# Function to preprocess text
def preprocess(text):
  	# Create Doc object
    doc = nlp(text, disable=['ner', 'parser'])
    # Generate lemmas
    lemmas = [token.lemma_ for token in doc]
    # Remove stopwords and non-alphabetic characters
    a_lemmas = [lemma for lemma in lemmas 
            if lemma.isalpha() and lemma not in stopwords]
    
    return ' '.join(a_lemmas)
  
# Apply preprocess to ted['transcript']
ted['transcript'] = ted['transcript'].apply(preprocess)
print(ted['transcript'])